In [1]:
!pip install musdb
!pip install librosa
# !pip install museval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: soundfile
    Found existing installation: soundfile 0.11.0
    Uninstalling soundfile-0.11.0:
      Successfully uninstalled soundfile-0.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wfdb 4.1.0 requires SoundFile<0.12.0,>=0.10.0, but you have soundfile 0.12.1 which is incompatible.


In [2]:
import os
import random
import math
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import musdb
import librosa
from tqdm.auto import tqdm

In [3]:
def audio_to_spectrogram(audio):
    spectrogram = np.abs(librosa.stft(audio, n_fft=512, hop_length=512, window='hann'))
    return spectrogram

In [4]:
class MUSDB18Dataset(Dataset):
    def __init__(self, musdb18_root, subset="train", split = "train", sr=44100, duration=6.0, seed = 42, transform = None):
        self.sr = sr
        self.duration = duration
        self.seed = seed
        self.transform = transform
        if(subset == 'test'):
            self.musdb18 = musdb.DB(root = musdb18_root, subsets=subset)
            print(f"Number of Testing Samples: {len(self.musdb18)}")
        else:
            self.musdb18 = musdb.DB(root = musdb18_root, subsets = subset, split = split)
            if split == "train":
                print(f"Number of Training Samples: {len(self.musdb18)}")
            else:
                print(f"Number of Validation Samples: {len(self.musdb18)}")
        
    def __getitem__(self, index):
        random.seed(self.seed)
        track = self.musdb18.tracks[index]
        track.chunk_duration = self.duration
        track.chunk_start = random.uniform(0, track.duration - track.chunk_duration)
        
        # Load the mixture waveform
        audio = track.audio.T

        # Load the target waveform (vocals)
        vocals = track.targets["vocals"].audio.T
        
        if self.transform:
            audio = self.transform(audio)
            vocals = self.transform(vocals)
            
        audio_tensor = torch.from_numpy(audio).float()
        vocals_tensor = torch.from_numpy(vocals).float()
        audio_tensor = audio_tensor.permute(0, 2, 1)
        vocals_tensor = vocals_tensor.permute(0, 2, 1)
        return audio_tensor, vocals_tensor

    def __len__(self):
        return len(self.musdb18.tracks)

In [5]:
def get_musdb18_dataloaders(root, batch_size=8, num_workers=0, seed = 42):
    train_set = MUSDB18Dataset(root, subset="train", transform = audio_to_spectrogram)
    val_set = MUSDB18Dataset(root, subset="train", split = 'valid', transform = audio_to_spectrogram)
    test_set = MUSDB18Dataset(root, subset="test", transform = audio_to_spectrogram)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, val_loader, test_loader

In [6]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, output_size, d_model = 512, num_heads = 4, num_layers = 3, dropout = 0.1):
        super().__init__()
        
        # input embedding layer
        self.input_embed = nn.Linear(input_size, d_model)

        # transformer encoder layers
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dropout=dropout)
        self.encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # output embedding layer
        self.output_embed = nn.Linear(d_model, output_size)
        
    def forward(self, x):
        # x is a tensor of shape (batch_size, num_channels, num_frames, num_bins)
        
        # reshape x to (num_frames, batch_size*num_channels, num_bins)
        x = x.permute(2, 0, 1, 3)
        x = x.reshape(x.size(0), -1, x.size(-1))
        
        # apply input embedding layer
        x = self.input_embed(x)
        
        # apply transformer encoder layers
        x = self.encoder(x)
        
        # apply output embedding layer
        x = self.output_embed(x)
        
        
        # reshape x back to (batch_size, num_channels, num_frames, num_bins)
        x = x.reshape(-1, 2, x.size(0), x.size(-1))
        return x

In [7]:
def train(model, dataloader, optimizer, loss_fn):
    model.train()
    total_loss = 0

    for i, (audio, vocals) in enumerate(dataloader):
        # move data to GPU if available
        if torch.cuda.is_available():
            audio = audio.cuda()
            vocals = vocals.cuda()

        # forward pass
        prediction = model(audio)
        loss = loss_fn(prediction, vocals)

        # backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update total loss
        total_loss += loss.item()

    # compute average loss
    avg_loss = total_loss / len(dataloader)

    return avg_loss

In [8]:
def validate(model, dataloader, loss_fn):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for i, (audio, vocals) in enumerate(dataloader):
            # move data to GPU if available
            if torch.cuda.is_available():
                audio = audio.cuda()
                vocals = vocals.cuda()

            # forward pass
            prediction = model(audio)
            loss = loss_fn(prediction, vocals)

            # update total loss
            total_loss += loss.item()

    # compute average loss
    avg_loss = total_loss / len(dataloader)

    return avg_loss

In [9]:
def fit(model, optimizer, loss_fn, num_epochs, train_dataloader, val_dataloader):
    for epoch in tqdm(range(num_epochs)):
        # train model for one epoch
        train_loss = train(model, train_dataloader, optimizer, loss_fn)
        print(f'Epoch {epoch + 1} - Train loss: {train_loss:.4f}')

        # evaluate model on validation data
        val_loss = validate(model, val_dataloader, loss_fn)
        print(f'Epoch {epoch + 1} - Val loss: {val_loss:.4f}')
        print("---------------------------------------------")

In [10]:
def calculate_sdr(predicted_output, ground_truth):
    """
    Calculates the Signal-to-Distortion Ratio (SDR) metric between a predicted output and its corresponding ground truth.

    Args:
    predicted_output: A numpy array of shape (number of channels, number of frames, number of bins).
    ground_truth: A numpy array of shape (number of channels, number of frames, number of bins).

    Returns:
    sdr: A scalar representing the SDR value between predicted_output and ground_truth.
    """
    eps = np.finfo(np.float32).eps  # To avoid division by zero errors
    num_channels = predicted_output.shape[0]
    sdr_sum = 0
    
#     print(predicted_output.shape, ground_truth.shape)
    
    for c in range(num_channels):
        # Compute the power of the true source signal
        true_source_power = np.sum(ground_truth[c]**2)

        # Compute the scalar product between true source signal and predicted signal
        true_pred_scalar = np.sum(ground_truth[c] * predicted_output[c])

        # Compute the SDR for this channel
        sdr = 10 * np.log10(true_source_power / (np.sum(ground_truth[c]**2) - true_pred_scalar + eps) + eps)
        if not math.isnan(sdr):
            sdr_sum += sdr

    # Compute the average SDR across all channels
    sdr = sdr_sum / num_channels

    return -sdr

In [11]:
def test(model, dataloader):
    model.eval()
    all_targets = []
    all_predictions = []
    sdr = 0
    count = 0
    with torch.no_grad():
        for i, (audio, vocals) in tqdm(enumerate(dataloader)):
            # move data to GPU if available
            if torch.cuda.is_available():
                audio = audio.cuda()
                vocals = vocals.cuda()

            # forward pass
            prediction = model(audio)
            
            prediction = prediction.permute(0, 1, 3, 2)
            vocals = vocals.permute(0, 1, 3, 2)

            # convert predictions and ground truth to numpy arrays
            prediction = prediction.cpu().numpy()
            vocals = vocals.cpu().numpy()
            
            for i in range(prediction.shape[0]):
                pred = prediction[i,:,:,:]
                targ = vocals[i,:,:,:]
                sdr += calculate_sdr(pred, targ)
                count += 1
    mean_sdr = sdr/count
    return mean_sdr

In [12]:
root = "/kaggle/input/musdb18/musdb18"
train_loader, val_loader, test_loader = get_musdb18_dataloaders(root, batch_size = 2)

Number of Training Samples: 87
Number of Validation Samples: 13
Number of Testing Samples: 50


In [13]:
# define model and optimizer
model = TransformerModel(input_size=257, output_size=257, num_heads = 8, num_layers = 6)
# model = PretrainedTransformer(output_size=257, num_heads = 8, num_layers = 6)
if torch.cuda.is_available():
    model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# define loss function
loss_fn = nn.MSELoss()

fit(model, optimizer, loss_fn, 250, train_loader, val_loader)

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1 - Train loss: 0.6966
Epoch 1 - Val loss: 0.5692
---------------------------------------------
Epoch 2 - Train loss: 0.6675
Epoch 2 - Val loss: 0.5490
---------------------------------------------
Epoch 3 - Train loss: 0.6510
Epoch 3 - Val loss: 0.5513
---------------------------------------------
Epoch 4 - Train loss: 0.6592
Epoch 4 - Val loss: 0.5694
---------------------------------------------
Epoch 5 - Train loss: 0.6694
Epoch 5 - Val loss: 0.5437
---------------------------------------------
Epoch 6 - Train loss: 0.6502
Epoch 6 - Val loss: 0.5425
---------------------------------------------
Epoch 7 - Train loss: 0.6472
Epoch 7 - Val loss: 0.5404
---------------------------------------------
Epoch 8 - Train loss: 0.6541
Epoch 8 - Val loss: 0.5515
---------------------------------------------
Epoch 9 - Train loss: 0.6399
Epoch 9 - Val loss: 0.5441
---------------------------------------------
Epoch 10 - Train loss: 0.6502
Epoch 10 - Val loss: 0.5327
------------------------

In [14]:
print(f"Training MSE: {validate(model, train_loader, loss_fn)}")
print(f"Validation MSE: {validate(model, val_loader, loss_fn)}")
print(f"Testing MSE: {validate(model, test_loader, loss_fn)}")

Training MSE: 0.5752765667540106
Validation MSE: 0.5254284994942802
Testing MSE: 0.44867627918720243


In [15]:
print(f"Training SDR: {test(model, train_loader)}")
print(f"Validation SDR: {test(model, val_loader)}")
print(f"Testing SDR: {test(model, test_loader)}")

0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: Runtime

Training SDR: 6.099803084400536


0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10


Validation SDR: 4.139887959911273


0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log10


Testing SDR: 2.7981035940349104
